# <font color=darkblue> Machine Learning model deployment with Flask framework</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with the help of the flask framework.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

### 2. Load the dataset

In [ ]:
df = pd.read_csv('car+data.csv')
df.head()

### 3. Check the shape and basic information of the dataset.

In [ ]:
df.shape

In [ ]:
df.info()

### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [ ]:
df[df.duplicated()]

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.duplicated().sum()

### 5. Drop the columns which you think redundant for the analysis.

In [ ]:
df.head()

In [ ]:
df.drop('Car_Name',axis=1, inplace=True)

In [ ]:
df.head()

### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [ ]:
df['age_of_the_car'] = 2024 - df['Year']

In [ ]:
df.drop('Year',axis=1, inplace=True)

In [ ]:
df.head()

### 7. Encode the categorical columns

In [ ]:
df.Transmission.unique()

In [ ]:
df.Seller_Type.unique()

In [ ]:
df.Fuel_Type.unique()

In [ ]:
df.head()

In [ ]:
df.Transmission.unique()

In [ ]:
df.Fuel_Type.unique()

In [ ]:
df.Seller_Type.unique()

### 8. Separate the target and independent features.

In [ ]:
X = df.drop('Selling_Price', axis=1)
y = df['Selling_Price']

### 9. Split the data into train and test.

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [ ]:
rf = RandomForestRegressor()
rf.fit(X_train,y_train)

In [ ]:
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print('r2-score train = ', r2_train)
print('r2-score test = ', r2_test)

### 11. Create a pickle file with an extension as .pkl

In [ ]:
import pickle
pickle.dump(rf, open('model.pkl','wb'))

# Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))

### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Car Price Prediction</title>

    <style>
        body {
            height: 100%;
            margin: 0;
            padding: 0;
            overflow: hidden;
            background-color: whitesmoke;
            font-family: Verdana, Geneva, Tahoma, sans-serif;
        }

        h1,
        h3 {
            color: rgb(0, 20, 160);
        }

        #main {
            width: 500px;
            margin: auto;
            margin-top: 100px;
        }

        input[type="text"],
        input[type="number"],
        textarea {
            width: 100%;
            padding: 10px;
            margin-bottom: 10px;
            border-radius: 5px;
            border: 1px solid rgb(0, 20, 160);
            font-weight: bold;
        }

        input:focus-within {
            background-color: rgb(191, 222, 255);
        }

        select {
            background-color: rgb(191, 222, 255);
            color: rgb(0, 0, 0);
            font-weight: bold;
            width: 50%;
            padding: 10px;
            margin-bottom: 10px;
            border-radius: 5px;
        }

        input[type="submit"] {
            background-color: rgb(56, 86, 255);
            color: rgb(255, 255, 255);
            padding: 10px 20px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-weight: bold;
        }

        input[type="submit"]:hover {
            background-color: rgb(0, 20, 160);
        }

        #predict {
            margin-top: 5%;
            width: 100%;
            text-align: center;
            color: rgb(0, 0, 0);
            font-size: larger;
            font-weight: bold;
        }

        label {
            color: rgb(0, 0, 128);
            font-weight: bold;
        }
    </style>

</head>

<body>
<div id="main">
    <h1>Enter Car Details</h1>
    <form action="{{ url_for('predict')}}" method="post">
        <input type="text" label="Name" name="Name" placeholder="Car Name" required>
        <input type="number" label='age_of_the_car' name="age_of_the_car" placeholder="Age of the Car (Years)" required>
        <input type="number" label='Present_Price' name="Present_Price" placeholder="Present Price (In Lakhs)"
               step="0.01" required>
        <input type="number" label='Kms_Driven' name="Kms_Driven" placeholder="Kms Driven" required>
        <select name="Fuel_Type" id="Fuel_Type" required>
            <option value="2">Petrol</option>
            <option value="1">Diesel</option>
            <option value="0">CNG</option>
        </select>
        <select name="Transmission" id="research" required="required">
            <option value="1">Manual</option>
            <option value="0">Automatic</option>
        </select>
        </br>
        </br>
        <input type="submit" value="Submit">
    </form>
</div>
<div id="predict">{{prediction_text}}</div>
</body>

</html>

### b) Create app.py file and write the predict function

In [ ]:
from flask import Flask, render_template, request
import pickle
import numpy as np
import sklearn

app = Flask(__name__)
model = pickle.load(open('model.pkl', 'rb'))


@app.route('/')
def index():
    return render_template('index.html')


@app.route("/predict", methods=['POST'])
def predict():
    if request.method == 'POST':
        car_name = request.form.get('Name')
        age_of_the_car = request.form['age_of_the_car']
        Present_Price = float(request.form['Present_Price'])
        Kms_Driven = int(request.form['Kms_Driven'])
        Fuel_Type = request.form['Fuel_Type']
        Transmission = request.form['Transmission']

        prediction = model.predict([[Present_Price, Kms_Driven, Fuel_Type, Transmission, age_of_the_car]])
        output = round(prediction[0], 2)
        prediction_text = 'Resale Value of your ' + car_name + ' is Rs. ' + str(output) + ' Lakhs.'
        return render_template('index.html', prediction_text=prediction_text)


if __name__ == "__main__":
    app.run(debug=True)


### 13. Run the app.py python file which will render to index html page then enter the input values and get the prediction.

In [ ]:
 * Running on http://127.0.0.1:5000

### Happy Learning :)